In [1]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, ResNet152V2  # Import ResNet152V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import classification_report

2023-08-26 14:08:05.896744: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 14:08:08.370367: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-08-26 14:08:08.370798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
# Load metadata CSV file
metadata = pd.read_csv('HAM10000_metadata.csv')

# Load images and labels
image_folder = 'HAM10000_Images'
image_size = (96, 96)


In [3]:
images = []
labels = []

for index, row in metadata.iterrows():
    image_path = os.path.join(image_folder, row['image_id'] + '.jpg')
    image = imread(image_path)
    resized_image = resize(image, image_size)
    images.append(resized_image)
    labels.append(row['dx'])


In [4]:
images = np.array(images)
labels = np.array(labels)

In [5]:
# Convert labels to categorical format
label_mapping = {label: idx for idx, label in enumerate(np.unique(labels))}
labels_encoded = np.array([label_mapping[label] for label in labels])
labels_categorical = to_categorical(labels_encoded, num_classes=len(label_mapping))

In [11]:
# Split the dataset into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels_categorical, test_size=0.3, random_state=42
)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.3, random_state=42
)


In [12]:

# Print the shape of each set
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Validation images shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Train images shape: (4907, 96, 96, 3)
Train labels shape: (4907, 7)
Validation images shape: (2103, 96, 96, 3)
Validation labels shape: (2103, 7)
Test images shape: (3005, 96, 96, 3)
Test labels shape: (3005, 7)


In [13]:
# Create an ImageDataGenerator instance for data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,  # Increase rotation range
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


In [14]:
# Apply data augmentation only to the training set
train_datagen = datagen.flow(train_images, train_labels, batch_size=32, shuffle=True)

***Resnet152v2***

In [15]:
# Load the pre-trained ResNet152V2 model (excluding top layers and with ImageNet weights)
base_model = ResNet152V2(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

2023-08-26 14:24:47.828453: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-08-26 14:24:47.828613: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-26 14:24:47.828700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-20230816-171017): /proc/driver/nvidia/version does not exist
2023-08-26 14:24:47.829367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow 

In [23]:
# Freeze the layers of the base model to retain pre-trained weights
fine_tune_at = 555
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [24]:
# Implement your specified model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 3, 3, 2048)        58331648  
                                                                 
 flatten_2 (Flatten)         (None, 18432)             0         
                                                                 
 batch_normalization_4 (Batc  (None, 18432)            73728     
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 128)               2359424   
                                                                 
 batch_normalization_5 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 128)              

In [30]:
# Unfreeze the last block of layers in the base model for fine-tuning
# fine_tune_at = -12
# for layer in model.layers[:fine_tune_at]:
#     layer.trainable = False
# for layer in model.layers[fine_tune_at:]:
#     layer.trainable = True


In [25]:
# Adjust the learning rate
def lr_scheduler(epoch):
    if epoch < 10:
        return 0.0001
    else:
        return 0.0001 * np.exp(0.1 * (10 - epoch))

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [26]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [28]:
# Train the model with the improved settings
epochs = 50
batch_size = 32

history = model.fit(
    train_datagen,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_images, val_labels),
    callbacks=[early_stopping, lr_schedule],
)


Epoch 1/50
154/154 [==============================] - 296s 2s/step - loss: 2.3707 - accuracy: 0.3405 - val_loss: 1.4892 - val_accuracy: 0.5873 - lr: 1.0000e-04
Epoch 2/50
154/154 [==============================] - 277s 2s/step - loss: 1.9578 - accuracy: 0.4638 - val_loss: 1.3804 - val_accuracy: 0.6429 - lr: 1.0000e-04
Epoch 3/50
154/154 [==============================] - 282s 2s/step - loss: 1.7481 - accuracy: 0.5266 - val_loss: 1.2881 - val_accuracy: 0.6743 - lr: 1.0000e-04
Epoch 4/50
154/154 [==============================] - 286s 2s/step - loss: 1.6316 - accuracy: 0.5578 - val_loss: 1.2688 - val_accuracy: 0.6876 - lr: 1.0000e-04
Epoch 5/50
154/154 [==============================] - 278s 2s/step - loss: 1.5213 - accuracy: 0.5773 - val_loss: 1.1840 - val_accuracy: 0.7071 - lr: 1.0000e-04
Epoch 6/50
154/154 [==============================] - 279s 2s/step - loss: 1.4042 - accuracy: 0.6110 - val_loss: 1.1939 - val_accuracy: 0.7042 - lr: 1.0000e-04
Epoch 7/50
154/154 [====================

In [29]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_images, val_labels, batch_size=batch_size)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

66/66 [==============================] - 81s 1s/step - loss: 1.1360 - accuracy: 0.7233
Validation Loss: 1.1360048055648804
Validation Accuracy: 0.7232524752616882


In [30]:
# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, batch_size=batch_size)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


94/94 [==============================] - 115s 1s/step - loss: 1.1205 - accuracy: 0.7012
Test Loss: 1.1205350160598755
Test Accuracy: 0.701164722442627


In [31]:
model.save('cnn_model-resnet152v2_96x96v1')

INFO:tensorflow:Assets written to: cnn_model-resnet152v2_96x96v1/assets


INFO:tensorflow:Assets written to: cnn_model-resnet152v2_96x96v1/assets
